In [ ]:
#import libraries
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

# Importing Deep Learning Libraries
import tensorflow as tf 
from tensorflow import keras
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

In [ ]:
train_dir = "/kaggle/input/fer2013/train/"
test_dir = "/kaggle/input/fer2013/test/"

# **Data Preprocessing**

In [ ]:
# Train data generator
datagen = ImageDataGenerator(rescale=1./255)

train_set = datagen.flow_from_directory(train_dir,
                                              target_size = (48,48),
                                              color_mode = "grayscale",
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True)
train_set.class_indices

In [ ]:
# Test data generator
val_datagen = ImageDataGenerator(rescale=1./255)

validation_set = val_datagen.flow_from_directory(test_dir,
                                              target_size = (48,48),
                                              color_mode = "grayscale",
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False)

validation_set.class_indices

# **Simple CNN model**

In [ ]:
# create model structure
emotion_model = Sequential()

# 1st CNN layer
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# 2nd CNN layer
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# Fully Connected layer
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

# compiling model
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [ ]:
emotion_model.summary()

In [ ]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_set,
        steps_per_epoch=28709 // 32,
        epochs=20,
        validation_data=validation_set,
        validation_steps=7178 // 32)

In [ ]:
# saving model
emotion_model.save('./simple_cnn_model.h5')

In [ ]:
history =  emotion_model_info.history
history.keys()

# **Model Evaluation**

In [ ]:
# ploting accuracy
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'] , c = "red", label='Validation Accuracy')
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(loc='lower right')
plt.savefig("./simple_cnn_accuracy.png")
plt.show()

In [ ]:
# ploting loss
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'] , c = "red", label='Validation Loss')
plt.title("Training loss vs Validation Loss")
plt.legend(loc='upper right')
plt.savefig("./simple_cnn_loss.png")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

# Predicting For Test Set
predictions = emotion_model.predict(validation_set)

# Printing Confution Matrix
cm = confusion_matrix(y_pred=np.argmax(predictions, axis=-1), y_true=validation_set.classes)                         
cm

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')# **Live Class Monitoring System(Face Emotion Recognition)**
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# compute predictions
predictions = emotion_model.predict_generator(generator=validation_set)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_set.classes
class_names = validation_set.class_indices.keys()

# compute confusion matrix
con_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot onfusion matrix
plt.figure()
plot_confusion_matrix(con_matrix, classes=class_names, title='Confusion matrix')
plt.savefig("./simple_cnn_cm.png")
plt.show()